In [1]:
pip install selenium


  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [62]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


1) create a driver
2) get / request
3) parse or extract desired info
4) save as csv or xml or ..

In [76]:
from selenium import webdriver
import csv
import requests
import re

In [69]:
driver = webdriver.Chrome()

In [71]:
driver.get("https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%2116225009011%2Cn%3A281407&ref=nav_em__nav_desktop_sa_intl_accessories_and_supplies_0_2_5_2")

In [72]:
blocks = driver.find_elements("xpath" , "//div[@class= 'a-section a-spacing-base']")

In [77]:
with open("amazon.csv", 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    header = ['Name', 'Price', 'Rating', 'Number of Ratings' , 'Photo URL']
    csv_writer.writerow(header)

    for block in blocks:
        #name
        product_name = block.find_element("xpath", ".//h2[@class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-4']").text
        
        #price
        price_whole = block.find_elements("xpath" , ".//span[@class='a-price-whole']")
        price_decimal = block.find_elements("xpath", ".//span[@class='a-price-fraction']")
        if price_whole != [] and price_decimal != []:
            price = '.'.join([price_whole[0].text, price_decimal[0].text])
        else:
            price = "Price not available"

        #ratings
        ratings_box = block.find_elements("xpath", './/div[@class="a-row a-size-small"]/span')
        if ratings_box != []:
            ratings = ratings_box[0].get_attribute('aria-label')
            ratings_num = ratings_box[1].get_attribute('aria-label')
        else:
            ratings, ratings_num = "Rating unavailable", "Number of ratings unavailable"


        #pic urls
        photo_url = block.find_element("xpath", ".//img[@class='s-image']").get_attribute('src')
            

        #print(product_name,"\n", price,"\n", ratings,"\n",ratings_num,"\n",photo_url, "\n\n\n\n")

        #writing into the csv file
        csv_writer.writerow([product_name,price,ratings,ratings_num,photo_url])


        #downloading photos
        response = requests.get(photo_url)

        product_name_updated = re.sub(r'\W+', '_', product_name)   #replace any nonword character with _ 
        img = f"{product_name_updated}.jpg"
        with open(img, 'wb') as img_file:
            img_file.write(response.content)


    print("Done")
    #csv file and photos are saved in the same directory as your script

Done
